In [57]:
# Data

#### TEXAS
# Import packages
import fredapi as fn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# When you don't know the data ID
# API Key
API_KEY = "c33d17d6ad546cfc58302d8906a42ece"

# Load Fred
fred = fn.Fred(API_KEY)

#   date-> Period of the data; realtime_start-> The first date the value is valid (when it was published); realtime_end-> The last date the value is valid (when there is no revisions published the previous data is done)..  
# Search FRED system for info by tag
"""
data_search = fred.search(text="Texas ",
                          order_by="seasonal_adjustment",sort_order="desc",
                         # filter=("frequency","Annual").
                         limit=100
                          )
                          


#ID_list=list(data_search.id)
data_1=data_search.sample(n=10)

#list(ID_list),ID_list

data_1["observation_start"]=pd.to_datetime(data_1["observation_start"])
data_1["observation_end"]=pd.to_datetime(data_1["observation_end"])
data_1["duration"]=data_1["observation_end"]-data_1["observation_start"]
data_1["duration"]=data_1["duration"].dt.days/365.
data_1["duration"]=data_1["duration"].astype(int)
data_1=data_1[data_1["duration"]>15]
data_2=data_1[data_1.isin(["M"]).any(axis=1)]
ID_list=list(data_2.id)
"""

# When you do know the data ID
# API Key

#ID of your data
ID_list=['GDPC1', 'UNRATE', 'CPILFESL', 'PCECC96', 'AAA', 'PSAVERT', 'FDEFX']

#Create dic of name:id
data_list={fred.search(i).title.iloc[0]+f" [Freq: {fred.search(i).frequency_short.iloc[0]}]":i for i in ID_list}

#Obtain df of ID_list
Data=pd.DataFrame({name+f" Id: [{tag}]":fred.get_series(tag,observation_start="1770",observation_end="2024") for name,tag in data_list.items()})
Data=Data.filter(like="[Freq: M]")

Data=Data.drop(columns=["Personal Saving Rate [Freq: M] Id: [PSAVERT]","Moody's Seasoned Aaa Corporate Bond Yield [Freq: M] Id: [AAA]"])
Data.head()

,Unemployment Rate [Freq: M] Id: [UNRATE],Consumer Price Index for All Urban Consumers: All Items Less Food and Energy in U.S. City Average [Freq: M] Id: [CPILFESL]
1919-01-01,NaN,NaN
1919-02-01,NaN,NaN
1919-03-01,NaN,NaN
1919-04-01,NaN,NaN
1919-05-01,NaN,NaN


In [59]:
# I think this is the best overall
#https://www.youtube.com/watch?v=nSkQy3ew3EI (good stuff)
from dataprep.eda import create_report,plot,plot_correlation,plot_missing,plot_diff,render,compute
from dataprep.datasets import load_dataset
import numpy as np
# Best and easiet to use
data=Data.dropna()
diff_data=data.diff().dropna()
log_diff_data=np.log(data).diff().dropna()
norm_data = (((data-data.mean()))/Data.std())

"""display_options_plots={
    "Bar Chart": "bar",
    "Pie Chart": "pie",
    "Word Cloud": "wordcloud",
    "Word Frequency": "wordfreq",
    "Word Length": "wordlen",
    "Histogram": "hist",
    "KDE Plot": "kde",
    "Normal Q-Q Plot": "qqnorm",
    "Box Plot": "box",
    "Line Chart": "line",
    "Stats": "stats",
    "Insights": "insight",
    "Scatter Plot": "scatter",
    "Hexbin Plot": "hexbin",
    "Nested Bar Chart": "nested",
    "Stacked Bar Chart": "stacked",
    "Heat Map": "heatmap",
    "Pearson": "pearson",
    "Spearman": "spearman",
    "KendallTau": "kendall",
    "Spectrum": "spectrum",
    "Dendrogram": "dendro",
    "PDF": "pdf",
    "CDF": "cdf",
    "Value Table": "value_table",
}"""
#plot(Data,Data.columns[20]).show_browser()
create_report(data,title=f"Data Report").show_browser()
#create_report(diff_data,title=f"Data Difference Report").show_browser()
#create_report(log_diff_data,title=f"Data Log Difference Report").show_browser()
#create_report(norm_data,title=f"Data Normalized Report").show_browser()

  0%|          | 0/432 [00:00<?, ?it/s]

c:\Users\zeroi\miniconda3\envs\econometrics\lib\site-packages\dask\core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [ ]:
#This works best on the web web. Is good, though i guess the 2 backwards is better
"""import dtale

# Assigning a reference to a running D-Tale process.
d = dtale.show(Data)

# Accessing data associated with D-Tale process.
tmp = d.data.copy()
tmp['d'] = 4

# Altering data associated with D-Tale process
# FYI: this will clear any front-end settings you have at the time for this process (filter, sorts, formatting)
#d.data = tmp This is kind of trashy

## Shutting down D-Tale process
#d.kill()

# Using Python's `webbrowser` package it will try and open your server's default browser to this process.
d.open_browser()

# There is also some helpful metadata about the process.
d._data_id  # The process's data identifier.

d._url  # The url to access the process.

d2 = dtale.get_instance(d._data_id)  # Returns a new reference to the instance running at that data_id.

dtale.instances()  # Prints a list of all ids & urls of running D-Tale sessions."""


In [ ]:
# Basura
#from ydata_profiling import ProfileReport


#Note: It appears that you must reset the index when doing this. Make sure just dont do it in place...
#profile = ProfileReport(Data.dropna().reset_index(),tsmode=True,orange_mode=True,title='Pandas Profiling Report') # Note: If tsmode True, we must get .dropna() on data
#profile.to_file("Texas_Analysis.html")

#profile.to_notebook_iframe()
#profile.to_notebook_iframe() # Appears to be the best choice we got here...

## Statsmodels.tsa


In [52]:
import statsmodels.tsa.api as smt
import statsmodels.api as sm
from IPython.display import display, Markdown,Latex
#Select the ARIMA model

model_1 = smt.ARIMA(data.iloc[:,[0]],order=(100,5,100)).fit(cov_type="robust")
model_1.summary()

c:\Users\zeroi\miniconda3\envs\ai\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


KeyboardInterrupt: 

In [141]:

from darts import TimeSeries
from darts.datasets import AirPassengersDataset
import darts.models.forecasting as dmf
from sklearn.model_selection import train_test_split
series = TimeSeries.from_dataframe(data)
series.freq,series.duration,series.has_datetime_index,series.is_deterministic,series.is_univariate,series.time_dim

# Split the data
RGDP = series[series.components[0]]
len(RGDP)
train, test = RGDP[:int(0.8*len(RGDP))], RGDP[int(0.8*len(RGDP)):]
#train.plot()
#test.plot()

# Build the model
model_2 = dmf.auto_arima.AutoARIMA()
model_2.fit(series[series.components[0]])
pred = model_2.predict(20)
RGDP.plot()
pred.plot(label="forecast", low_quantile=0.05, high_quantile=0.95)
plt.legend()

AttributeError: module 'matplotlib.cbook' has no attribute '_Stack'

AttributeError: 'super' object has no attribute '__getattr__'

AttributeError: 'super' object has no attribute '__getattr__'